# Stage 1 Model Selection: Summary of Results

Model selection was conducted using a method of time series cross-validation called rolling forecast origin. To avoid leakage of future observations, the method incrementally moves the forecast origin forward in time and then makes a prediction. For each new fold we implemented a stride of seven days. We performed a two stage model selection procedure. In the first stage, we used an aggregate regional level time series to screen and identify the most promising candidate models for up to a 365 day forecast (27 folds).

This notebook generates the summary result tables and charts for stage 1 of model selection (screening)


In [1]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [2]:
cwd = os.getcwd()
cwd

'/home/tom/Documents/code/swast-benchmarking/results'

In [3]:
if cwd[-7:] != "results":
    mypath = './results/model_selection/stage1/'
    TABLE_PATH = './paper/tables/'
    FIGURE_PATH = './paper/figures/'
    APPENDIX_PATH = './paper/appendix/'
else:
    mypath = './model_selection/stage1/'
    TABLE_PATH = '../paper/tables/'
    FIGURE_PATH = '../paper/figures/'
    APPENDIX_PATH = '../paper/appendix/'

In [4]:
result_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
results_mean = pd.DataFrame()
results_med = pd.DataFrame()
results_mean_std = pd.DataFrame()

## Point Estimate Results

In [6]:
error_measures = ['smape', 'rmse', 'mase', 'coverage_80', 'coverage_95']

In [7]:
for metric in error_measures:
    to_read = [filename for filename in result_files if metric in filename]
    model_names = [name[:name.index('_')] for name in to_read]
    
    for filename, model_name in zip(to_read, model_names):
        df = pd.read_csv(mypath + filename, index_col=0)

        prefix = model_name + '_' + metric
        results_mean[prefix + '_mean'] = df.mean()
        results_mean[prefix  + '_std'] = df.std()
        results_med[prefix + '_med'] = df.median()
        results_med[prefix + '_iqr'] = df.quantile(0.75) - df.quantile(0.25)
        
        results_mean_std[prefix] = results_mean[prefix + '_mean'].map('{:,.2f}'.format) \
            + ' (' + results_mean[prefix  + '_std'].map('{:,.2f}'.format) + ')'

In [8]:
results_mean.filter(like="fbp")

,Trust-fbp-arima_smape_mean,Trust-fbp-arima_smape_std,Trust-fbp-arima-hw_smape_mean,Trust-fbp-arima-hw_smape_std,Trust-fbp_smape_mean,Trust-fbp_smape_std,Trust-fbp-arima_rmse_mean,Trust-fbp-arima_rmse_std,Trust-fbp_rmse_mean,Trust-fbp_rmse_std,...,Trust-fbp-arima-hw_coverage_80_mean,Trust-fbp-arima-hw_coverage_80_std,Trust-fbp-arima_coverage_80_mean,Trust-fbp-arima_coverage_80_std,Trust-fbp-arima-hw_coverage_95_mean,Trust-fbp-arima-hw_coverage_95_std,Trust-fbp_coverage_95_mean,Trust-fbp_coverage_95_std,Trust-fbp-arima_coverage_95_mean,Trust-fbp-arima_coverage_95_std
7,2.888748,1.154105,2.846749,1.163151,2.992764,1.453381,76.322438,31.861061,81.477287,45.993262,...,0.820106,0.200441,0.788360,0.214726,0.936508,0.127365,0.915344,0.138727,0.936508,0.127365
14,3.069071,1.078228,3.092036,1.165531,3.054805,1.252905,82.961792,31.404482,86.065006,41.350955,...,0.812169,0.190171,0.775132,0.181248,0.920635,0.137728,0.912698,0.114377,0.928571,0.112066
21,3.142560,0.854233,3.182474,0.999269,3.077355,1.008357,87.044831,27.738723,88.932570,36.300717,...,0.821869,0.157155,0.776014,0.141517,0.929453,0.110001,0.913580,0.092485,0.924162,0.096384
28,3.181019,0.628019,3.240986,0.803257,3.085396,0.817484,89.581903,24.119081,90.760518,32.138012,...,0.833333,0.129150,0.781746,0.109109,0.937831,0.090020,0.914021,0.078158,0.921958,0.082896
35,3.243412,0.546131,3.332646,0.749498,3.098144,0.727439,92.174299,22.037625,92.251442,28.970328,...,0.834921,0.125127,0.774603,0.101583,0.940741,0.078816,0.913228,0.071530,0.920635,0.073201
42,3.307821,0.517146,3.423800,0.735622,3.115493,0.678602,94.456973,20.510388,93.563846,26.261380,...,0.836861,0.122425,0.768078,0.097409,0.944444,0.069259,0.910935,0.067857,0.919753,0.064414
49,3.363752,0.488710,3.494352,0.724623,3.138307,0.630983,96.489349,19.224001,94.933570,23.533562,...,0.837491,0.108209,0.764928,0.090275,0.945578,0.062775,0.908541,0.064131,0.919123,0.056171
56,3.419441,0.440075,3.565179,0.677578,3.163085,0.580087,98.388413,17.644049,96.359590,20.534614,...,0.843254,0.095310,0.761905,0.079552,0.947090,0.056574,0.906746,0.059294,0.917328,0.048310
63,3.480343,0.420162,3.647479,0.677669,3.191124,0.543042,100.425831,15.985038,97.673905,17.567544,...,0.844209,0.093910,0.753674,0.077808,0.948854,0.052522,0.904762,0.056206,0.916520,0.041902
70,3.522194,0.391239,3.698958,0.614217,3.216202,0.526889,101.982125,13.918642,98.775720,15.222167,...,0.849206,0.085286,0.747090,0.070300,0.950794,0.046937,0.902116,0.054428,0.916402,0.035814


## Formatted results

In [9]:
point_forecasts = pd.concat([results_mean_std.filter(like='smape'), 
                             results_mean_std.filter(like='rmse'),
                             results_mean_std.filter(like='mase')], axis=1)

columns = point_forecasts.columns
columns = [s.replace('Trust-', '') for s in columns]
point_forecasts.columns = columns

In [10]:
pf_smape = point_forecasts.filter(like='smape')
pf_smape.reindex(sorted(pf_smape.columns), axis=1)

,ar_smape,arima-hw_smape,arima_smape,comb_smape,elastic-net_smape,fbp-arima-hw_smape,fbp-arima_smape,fbp_smape,harmonic_smape,hw_smape,reg-arima2_smape,reg-arima_smape,snaive_smape,ssa_smape,tbats_smape
7,3.21 (1.12),2.84 (1.09),3.04 (1.26),4.14 (1.49),3.26 (1.20),2.85 (1.16),2.89 (1.15),2.99 (1.45),3.23 (1.50),2.87 (1.24),4.57 (2.49),2.82 (0.96),3.48 (1.19),3.73 (1.57),3.12 (2.04)
14,3.56 (1.05),3.19 (1.19),3.39 (1.28),4.39 (1.43),3.63 (1.06),3.09 (1.17),3.07 (1.08),3.05 (1.25),3.59 (1.45),3.23 (1.44),6.10 (3.13),3.16 (0.96),3.94 (1.32),3.96 (1.44),3.48 (2.16)
21,3.80 (0.88),3.35 (1.15),3.58 (1.19),4.48 (1.33),3.85 (0.89),3.18 (1.00),3.14 (0.85),3.08 (1.01),3.79 (1.22),3.39 (1.40),7.04 (3.46),3.38 (0.95),4.12 (1.38),4.16 (1.38),3.71 (2.36)
28,4.00 (0.86),3.46 (1.06),3.73 (1.14),4.57 (1.16),4.07 (0.83),3.24 (0.80),3.18 (0.63),3.09 (0.82),3.96 (1.14),3.50 (1.30),7.80 (3.65),3.53 (0.88),4.19 (1.33),4.29 (1.38),3.85 (2.37)
35,4.16 (0.96),3.61 (1.10),3.87 (1.18),4.71 (1.21),4.24 (0.91),3.33 (0.75),3.24 (0.55),3.10 (0.73),4.15 (1.39),3.66 (1.31),8.51 (3.99),3.70 (0.94),4.35 (1.34),4.40 (1.40),4.02 (2.41)
42,4.30 (1.03),3.75 (1.13),3.98 (1.19),4.83 (1.24),4.37 (0.94),3.42 (0.74),3.31 (0.52),3.12 (0.68),4.31 (1.48),3.82 (1.32),9.18 (4.21),3.85 (0.99),4.51 (1.33),4.47 (1.44),4.19 (2.44)
49,4.43 (1.07),3.85 (1.14),4.07 (1.17),4.91 (1.18),4.47 (0.94),3.49 (0.72),3.36 (0.49),3.14 (0.63),4.44 (1.49),3.94 (1.31),9.78 (4.20),3.98 (1.02),4.61 (1.32),4.52 (1.44),4.29 (2.43)
56,4.57 (1.11),3.94 (1.11),4.14 (1.12),5.00 (1.14),4.57 (0.91),3.57 (0.68),3.42 (0.44),3.16 (0.58),4.58 (1.48),4.03 (1.28),10.32 (3.99),4.09 (0.98),4.67 (1.33),4.59 (1.41),4.36 (2.39)
63,4.67 (1.12),4.04 (1.11),4.22 (1.09),5.12 (1.14),4.65 (0.90),3.65 (0.68),3.48 (0.42),3.19 (0.54),4.70 (1.52),4.14 (1.30),10.83 (3.75),4.20 (1.05),4.77 (1.36),4.66 (1.35),4.44 (2.33)
70,4.78 (1.15),4.11 (1.01),4.27 (1.02),5.21 (1.08),4.72 (0.90),3.70 (0.61),3.52 (0.39),3.22 (0.53),4.81 (1.54),4.21 (1.20),11.25 (3.43),4.29 (1.06),4.82 (1.26),4.73 (1.27),4.49 (2.29)


In [11]:
pf_rmse = point_forecasts.filter(like='rmse')
pf_rmse.reindex(sorted(pf_rmse.columns), axis=1)

,ar_rmse,arima-hw_rmse,arima_rmse,comb_rmse,elastic-net_rmse,fbp-arima-hw_rmse,fbp-arima_rmse,fbp_rmse,harmonic_rmse,hw_rmse,reg-arima2_rmse,reg-arima_rmse,snaive_rmse,ssa_rmse,tbats_rmse
7,83.47 (28.06),74.99 (28.43),81.33 (38.91),116.03 (42.17),84.67 (29.29),75.59 (32.44),76.32 (31.86),81.48 (45.99),87.40 (47.60),77.22 (36.81),114.16 (56.82),74.38 (25.07),96.91 (37.11),99.68 (45.75),82.98 (52.42)
14,94.28 (28.84),85.53 (33.14),92.78 (41.97),124.74 (42.20),95.80 (29.17),83.44 (34.35),82.96 (31.40),86.07 (41.35),98.80 (46.27),88.15 (42.64),156.88 (83.87),84.99 (26.96),109.71 (41.88),107.77 (43.49),94.06 (57.13)
21,101.66 (27.03),91.41 (33.06),99.88 (41.13),130.02 (39.50),103.26 (26.46),88.01 (31.60),87.04 (27.74),88.93 (36.30),106.11 (42.47),94.07 (41.97),183.17 (93.47),91.97 (26.99),115.38 (42.94),114.16 (41.94),102.13 (60.69)
28,107.53 (27.05),95.45 (31.45),104.92 (40.30),133.74 (36.56),109.28 (24.95),90.99 (28.15),89.58 (24.12),90.76 (32.14),111.98 (39.48),98.20 (39.83),202.79 (96.62),96.63 (25.82),118.34 (42.92),118.58 (41.22),106.99 (60.51)
35,111.63 (28.11),100.07 (32.11),109.55 (40.69),138.37 (37.22),113.65 (25.44),94.46 (26.64),92.17 (22.04),92.25 (28.97),117.35 (43.46),103.50 (39.64),220.22 (103.31),101.42 (26.70),123.26 (42.33),122.19 (40.95),112.48 (60.47)
42,115.04 (28.81),104.24 (32.24),113.36 (40.41),142.34 (37.96),116.98 (25.21),97.57 (25.54),94.46 (20.51),93.56 (26.26),122.15 (44.74),108.35 (39.09),236.55 (106.99),105.73 (27.14),128.11 (40.34),125.00 (41.14),117.29 (60.29)
49,118.08 (29.16),107.46 (32.54),116.52 (39.75),145.74 (36.03),119.60 (24.46),100.10 (24.73),96.49 (19.22),94.93 (23.53),126.17 (44.46),111.91 (38.72),251.12 (106.16),109.54 (27.64),131.40 (39.79),127.36 (41.05),120.61 (59.59)
56,121.28 (29.37),110.35 (32.22),119.28 (38.52),148.69 (34.95),122.09 (23.40),102.43 (23.65),98.39 (17.64),96.36 (20.53),130.32 (43.79),114.95 (38.35),264.48 (100.54),112.77 (26.88),133.95 (40.20),129.83 (40.53),123.02 (58.78)
63,123.88 (29.01),113.67 (31.90),122.31 (37.26),152.25 (33.77),123.82 (23.11),105.04 (22.70),100.43 (15.99),97.67 (17.57),133.95 (43.86),118.45 (38.40),277.97 (93.92),116.08 (28.47),136.59 (40.30),132.14 (39.44),125.67 (57.53)
70,126.50 (29.10),116.20 (29.55),124.65 (35.42),154.95 (31.58),125.69 (23.23),106.97 (20.36),101.98 (13.92),98.78 (15.22),137.34 (43.87),121.07 (35.65),289.23 (83.74),118.89 (28.51),138.70 (37.25),134.65 (37.87),127.89 (55.82)


In [12]:
pf_mase = point_forecasts.filter(like='mase')
pf_mase.reindex(sorted(pf_mase.columns), axis=1)

,ar_mase,arima-hw_mase,arima_mase,comb_mase,elastic-net_mase,fbp-arima-hw_mase,fbp-arima_mase,fbp_mase,harmonic_mase,hw_mase,reg-arima2_mase,reg-arima_mase,snaive_mase,ssa_mase,tbats_mase
7,0.86 (0.32),0.77 (0.33),0.82 (0.39),1.12 (0.45),0.88 (0.34),0.77 (0.35),0.78 (0.34),0.81 (0.42),0.88 (0.46),0.78 (0.37),1.21 (0.67),0.76 (0.28),0.94 (0.35),1.01 (0.46),0.85 (0.59)
14,0.96 (0.31),0.87 (0.36),0.92 (0.40),1.19 (0.44),0.97 (0.31),0.84 (0.35),0.83 (0.32),0.82 (0.36),0.97 (0.44),0.88 (0.43),1.62 (0.90),0.85 (0.29),1.06 (0.39),1.07 (0.42),0.95 (0.63)
21,1.02 (0.27),0.91 (0.35),0.97 (0.37),1.22 (0.41),1.03 (0.26),0.86 (0.30),0.85 (0.26),0.83 (0.29),1.03 (0.38),0.92 (0.42),1.88 (1.02),0.91 (0.29),1.12 (0.41),1.13 (0.40),1.01 (0.68)
28,1.07 (0.27),0.94 (0.33),1.01 (0.36),1.24 (0.37),1.09 (0.25),0.88 (0.25),0.86 (0.20),0.83 (0.23),1.07 (0.35),0.95 (0.39),2.08 (1.09),0.96 (0.28),1.14 (0.40),1.16 (0.39),1.05 (0.68)
35,1.12 (0.29),0.98 (0.34),1.05 (0.37),1.28 (0.37),1.14 (0.27),0.91 (0.23),0.88 (0.17),0.84 (0.20),1.12 (0.41),1.00 (0.40),2.27 (1.17),1.00 (0.29),1.18 (0.40),1.19 (0.40),1.10 (0.69)
42,1.15 (0.31),1.02 (0.34),1.08 (0.37),1.31 (0.38),1.17 (0.27),0.93 (0.23),0.90 (0.17),0.84 (0.18),1.17 (0.43),1.04 (0.40),2.44 (1.23),1.05 (0.31),1.23 (0.40),1.21 (0.40),1.15 (0.69)
49,1.19 (0.32),1.05 (0.35),1.11 (0.36),1.34 (0.36),1.20 (0.27),0.95 (0.22),0.91 (0.16),0.85 (0.16),1.20 (0.43),1.08 (0.40),2.60 (1.22),1.08 (0.31),1.25 (0.39),1.23 (0.40),1.18 (0.69)
56,1.22 (0.32),1.07 (0.34),1.13 (0.34),1.36 (0.34),1.23 (0.27),0.97 (0.21),0.93 (0.14),0.86 (0.15),1.24 (0.43),1.10 (0.38),2.74 (1.16),1.11 (0.30),1.27 (0.39),1.25 (0.40),1.19 (0.68)
63,1.25 (0.33),1.10 (0.33),1.15 (0.33),1.39 (0.34),1.25 (0.26),0.99 (0.21),0.95 (0.13),0.87 (0.13),1.27 (0.43),1.13 (0.39),2.87 (1.10),1.14 (0.31),1.30 (0.40),1.27 (0.38),1.22 (0.66)
70,1.28 (0.33),1.12 (0.30),1.17 (0.31),1.42 (0.32),1.27 (0.26),1.01 (0.19),0.96 (0.12),0.87 (0.12),1.30 (0.44),1.15 (0.36),2.97 (1.01),1.17 (0.31),1.31 (0.37),1.29 (0.36),1.23 (0.65)


In [13]:
columns = pf_rmse.columns
columns = [s.replace('_rmse', '') for s in columns]
pf_rmse.columns = columns
pf_rmse.sort_index(axis=1)

,ar,arima,arima-hw,comb,elastic-net,fbp,fbp-arima,fbp-arima-hw,harmonic,hw,reg-arima,reg-arima2,snaive,ssa,tbats
7,83.47 (28.06),81.33 (38.91),74.99 (28.43),116.03 (42.17),84.67 (29.29),81.48 (45.99),76.32 (31.86),75.59 (32.44),87.40 (47.60),77.22 (36.81),74.38 (25.07),114.16 (56.82),96.91 (37.11),99.68 (45.75),82.98 (52.42)
14,94.28 (28.84),92.78 (41.97),85.53 (33.14),124.74 (42.20),95.80 (29.17),86.07 (41.35),82.96 (31.40),83.44 (34.35),98.80 (46.27),88.15 (42.64),84.99 (26.96),156.88 (83.87),109.71 (41.88),107.77 (43.49),94.06 (57.13)
21,101.66 (27.03),99.88 (41.13),91.41 (33.06),130.02 (39.50),103.26 (26.46),88.93 (36.30),87.04 (27.74),88.01 (31.60),106.11 (42.47),94.07 (41.97),91.97 (26.99),183.17 (93.47),115.38 (42.94),114.16 (41.94),102.13 (60.69)
28,107.53 (27.05),104.92 (40.30),95.45 (31.45),133.74 (36.56),109.28 (24.95),90.76 (32.14),89.58 (24.12),90.99 (28.15),111.98 (39.48),98.20 (39.83),96.63 (25.82),202.79 (96.62),118.34 (42.92),118.58 (41.22),106.99 (60.51)
35,111.63 (28.11),109.55 (40.69),100.07 (32.11),138.37 (37.22),113.65 (25.44),92.25 (28.97),92.17 (22.04),94.46 (26.64),117.35 (43.46),103.50 (39.64),101.42 (26.70),220.22 (103.31),123.26 (42.33),122.19 (40.95),112.48 (60.47)
42,115.04 (28.81),113.36 (40.41),104.24 (32.24),142.34 (37.96),116.98 (25.21),93.56 (26.26),94.46 (20.51),97.57 (25.54),122.15 (44.74),108.35 (39.09),105.73 (27.14),236.55 (106.99),128.11 (40.34),125.00 (41.14),117.29 (60.29)
49,118.08 (29.16),116.52 (39.75),107.46 (32.54),145.74 (36.03),119.60 (24.46),94.93 (23.53),96.49 (19.22),100.10 (24.73),126.17 (44.46),111.91 (38.72),109.54 (27.64),251.12 (106.16),131.40 (39.79),127.36 (41.05),120.61 (59.59)
56,121.28 (29.37),119.28 (38.52),110.35 (32.22),148.69 (34.95),122.09 (23.40),96.36 (20.53),98.39 (17.64),102.43 (23.65),130.32 (43.79),114.95 (38.35),112.77 (26.88),264.48 (100.54),133.95 (40.20),129.83 (40.53),123.02 (58.78)
63,123.88 (29.01),122.31 (37.26),113.67 (31.90),152.25 (33.77),123.82 (23.11),97.67 (17.57),100.43 (15.99),105.04 (22.70),133.95 (43.86),118.45 (38.40),116.08 (28.47),277.97 (93.92),136.59 (40.30),132.14 (39.44),125.67 (57.53)
70,126.50 (29.10),124.65 (35.42),116.20 (29.55),154.95 (31.58),125.69 (23.23),98.78 (15.22),101.98 (13.92),106.97 (20.36),137.34 (43.87),121.07 (35.65),118.89 (28.51),289.23 (83.74),138.70 (37.25),134.65 (37.87),127.89 (55.82)


In [14]:
ordered_cols = ['snaive', 'hw', 'arima', 'ar', 'reg-arima', 'reg-arima2', 
           'elastic-net', 'harmonic', 'ssa', 'fbp', 'tbats', 'comb', 'arima-hw',
           'fbp-arima', 'fbp-arima-hw']

In [15]:
columns = pf_smape.columns
columns = [s.replace('_smape', '') for s in columns]
pf_smape.columns = columns
pf_smape.sort_index(axis=1)

,ar,arima,arima-hw,comb,elastic-net,fbp,fbp-arima,fbp-arima-hw,harmonic,hw,reg-arima,reg-arima2,snaive,ssa,tbats
7,3.21 (1.12),3.04 (1.26),2.84 (1.09),4.14 (1.49),3.26 (1.20),2.99 (1.45),2.89 (1.15),2.85 (1.16),3.23 (1.50),2.87 (1.24),2.82 (0.96),4.57 (2.49),3.48 (1.19),3.73 (1.57),3.12 (2.04)
14,3.56 (1.05),3.39 (1.28),3.19 (1.19),4.39 (1.43),3.63 (1.06),3.05 (1.25),3.07 (1.08),3.09 (1.17),3.59 (1.45),3.23 (1.44),3.16 (0.96),6.10 (3.13),3.94 (1.32),3.96 (1.44),3.48 (2.16)
21,3.80 (0.88),3.58 (1.19),3.35 (1.15),4.48 (1.33),3.85 (0.89),3.08 (1.01),3.14 (0.85),3.18 (1.00),3.79 (1.22),3.39 (1.40),3.38 (0.95),7.04 (3.46),4.12 (1.38),4.16 (1.38),3.71 (2.36)
28,4.00 (0.86),3.73 (1.14),3.46 (1.06),4.57 (1.16),4.07 (0.83),3.09 (0.82),3.18 (0.63),3.24 (0.80),3.96 (1.14),3.50 (1.30),3.53 (0.88),7.80 (3.65),4.19 (1.33),4.29 (1.38),3.85 (2.37)
35,4.16 (0.96),3.87 (1.18),3.61 (1.10),4.71 (1.21),4.24 (0.91),3.10 (0.73),3.24 (0.55),3.33 (0.75),4.15 (1.39),3.66 (1.31),3.70 (0.94),8.51 (3.99),4.35 (1.34),4.40 (1.40),4.02 (2.41)
42,4.30 (1.03),3.98 (1.19),3.75 (1.13),4.83 (1.24),4.37 (0.94),3.12 (0.68),3.31 (0.52),3.42 (0.74),4.31 (1.48),3.82 (1.32),3.85 (0.99),9.18 (4.21),4.51 (1.33),4.47 (1.44),4.19 (2.44)
49,4.43 (1.07),4.07 (1.17),3.85 (1.14),4.91 (1.18),4.47 (0.94),3.14 (0.63),3.36 (0.49),3.49 (0.72),4.44 (1.49),3.94 (1.31),3.98 (1.02),9.78 (4.20),4.61 (1.32),4.52 (1.44),4.29 (2.43)
56,4.57 (1.11),4.14 (1.12),3.94 (1.11),5.00 (1.14),4.57 (0.91),3.16 (0.58),3.42 (0.44),3.57 (0.68),4.58 (1.48),4.03 (1.28),4.09 (0.98),10.32 (3.99),4.67 (1.33),4.59 (1.41),4.36 (2.39)
63,4.67 (1.12),4.22 (1.09),4.04 (1.11),5.12 (1.14),4.65 (0.90),3.19 (0.54),3.48 (0.42),3.65 (0.68),4.70 (1.52),4.14 (1.30),4.20 (1.05),10.83 (3.75),4.77 (1.36),4.66 (1.35),4.44 (2.33)
70,4.78 (1.15),4.27 (1.02),4.11 (1.01),5.21 (1.08),4.72 (0.90),3.22 (0.53),3.52 (0.39),3.70 (0.61),4.81 (1.54),4.21 (1.20),4.29 (1.06),11.25 (3.43),4.82 (1.26),4.73 (1.27),4.49 (2.29)


In [16]:
columns = pf_mase.columns
columns = [s.replace('_mase', '') for s in columns]
pf_mase.columns = columns
pf_mase.sort_index(axis=1)[ordered_cols]

,snaive,hw,arima,ar,reg-arima,reg-arima2,elastic-net,harmonic,ssa,fbp,tbats,comb,arima-hw,fbp-arima,fbp-arima-hw
7,0.94 (0.35),0.78 (0.37),0.82 (0.39),0.86 (0.32),0.76 (0.28),1.21 (0.67),0.88 (0.34),0.88 (0.46),1.01 (0.46),0.81 (0.42),0.85 (0.59),1.12 (0.45),0.77 (0.33),0.78 (0.34),0.77 (0.35)
14,1.06 (0.39),0.88 (0.43),0.92 (0.40),0.96 (0.31),0.85 (0.29),1.62 (0.90),0.97 (0.31),0.97 (0.44),1.07 (0.42),0.82 (0.36),0.95 (0.63),1.19 (0.44),0.87 (0.36),0.83 (0.32),0.84 (0.35)
21,1.12 (0.41),0.92 (0.42),0.97 (0.37),1.02 (0.27),0.91 (0.29),1.88 (1.02),1.03 (0.26),1.03 (0.38),1.13 (0.40),0.83 (0.29),1.01 (0.68),1.22 (0.41),0.91 (0.35),0.85 (0.26),0.86 (0.30)
28,1.14 (0.40),0.95 (0.39),1.01 (0.36),1.07 (0.27),0.96 (0.28),2.08 (1.09),1.09 (0.25),1.07 (0.35),1.16 (0.39),0.83 (0.23),1.05 (0.68),1.24 (0.37),0.94 (0.33),0.86 (0.20),0.88 (0.25)
35,1.18 (0.40),1.00 (0.40),1.05 (0.37),1.12 (0.29),1.00 (0.29),2.27 (1.17),1.14 (0.27),1.12 (0.41),1.19 (0.40),0.84 (0.20),1.10 (0.69),1.28 (0.37),0.98 (0.34),0.88 (0.17),0.91 (0.23)
42,1.23 (0.40),1.04 (0.40),1.08 (0.37),1.15 (0.31),1.05 (0.31),2.44 (1.23),1.17 (0.27),1.17 (0.43),1.21 (0.40),0.84 (0.18),1.15 (0.69),1.31 (0.38),1.02 (0.34),0.90 (0.17),0.93 (0.23)
49,1.25 (0.39),1.08 (0.40),1.11 (0.36),1.19 (0.32),1.08 (0.31),2.60 (1.22),1.20 (0.27),1.20 (0.43),1.23 (0.40),0.85 (0.16),1.18 (0.69),1.34 (0.36),1.05 (0.35),0.91 (0.16),0.95 (0.22)
56,1.27 (0.39),1.10 (0.38),1.13 (0.34),1.22 (0.32),1.11 (0.30),2.74 (1.16),1.23 (0.27),1.24 (0.43),1.25 (0.40),0.86 (0.15),1.19 (0.68),1.36 (0.34),1.07 (0.34),0.93 (0.14),0.97 (0.21)
63,1.30 (0.40),1.13 (0.39),1.15 (0.33),1.25 (0.33),1.14 (0.31),2.87 (1.10),1.25 (0.26),1.27 (0.43),1.27 (0.38),0.87 (0.13),1.22 (0.66),1.39 (0.34),1.10 (0.33),0.95 (0.13),0.99 (0.21)
70,1.31 (0.37),1.15 (0.36),1.17 (0.31),1.28 (0.33),1.17 (0.31),2.97 (1.01),1.27 (0.26),1.30 (0.44),1.29 (0.36),0.87 (0.12),1.23 (0.65),1.42 (0.32),1.12 (0.30),0.96 (0.12),1.01 (0.19)


In [17]:
model_nos = [str(i) for i in range(15)]
model_nos[0] = 'naive'

In [18]:
pf_mase2 = pf_mase[ordered_cols]
pf_mase2.columns = model_nos
pf_mase2

,naive,1,2,3,4,5,6,7,8,9,10,11,12,13,14
7,0.94 (0.35),0.78 (0.37),0.82 (0.39),0.86 (0.32),0.76 (0.28),1.21 (0.67),0.88 (0.34),0.88 (0.46),1.01 (0.46),0.81 (0.42),0.85 (0.59),1.12 (0.45),0.77 (0.33),0.78 (0.34),0.77 (0.35)
14,1.06 (0.39),0.88 (0.43),0.92 (0.40),0.96 (0.31),0.85 (0.29),1.62 (0.90),0.97 (0.31),0.97 (0.44),1.07 (0.42),0.82 (0.36),0.95 (0.63),1.19 (0.44),0.87 (0.36),0.83 (0.32),0.84 (0.35)
21,1.12 (0.41),0.92 (0.42),0.97 (0.37),1.02 (0.27),0.91 (0.29),1.88 (1.02),1.03 (0.26),1.03 (0.38),1.13 (0.40),0.83 (0.29),1.01 (0.68),1.22 (0.41),0.91 (0.35),0.85 (0.26),0.86 (0.30)
28,1.14 (0.40),0.95 (0.39),1.01 (0.36),1.07 (0.27),0.96 (0.28),2.08 (1.09),1.09 (0.25),1.07 (0.35),1.16 (0.39),0.83 (0.23),1.05 (0.68),1.24 (0.37),0.94 (0.33),0.86 (0.20),0.88 (0.25)
35,1.18 (0.40),1.00 (0.40),1.05 (0.37),1.12 (0.29),1.00 (0.29),2.27 (1.17),1.14 (0.27),1.12 (0.41),1.19 (0.40),0.84 (0.20),1.10 (0.69),1.28 (0.37),0.98 (0.34),0.88 (0.17),0.91 (0.23)
42,1.23 (0.40),1.04 (0.40),1.08 (0.37),1.15 (0.31),1.05 (0.31),2.44 (1.23),1.17 (0.27),1.17 (0.43),1.21 (0.40),0.84 (0.18),1.15 (0.69),1.31 (0.38),1.02 (0.34),0.90 (0.17),0.93 (0.23)
49,1.25 (0.39),1.08 (0.40),1.11 (0.36),1.19 (0.32),1.08 (0.31),2.60 (1.22),1.20 (0.27),1.20 (0.43),1.23 (0.40),0.85 (0.16),1.18 (0.69),1.34 (0.36),1.05 (0.35),0.91 (0.16),0.95 (0.22)
56,1.27 (0.39),1.10 (0.38),1.13 (0.34),1.22 (0.32),1.11 (0.30),2.74 (1.16),1.23 (0.27),1.24 (0.43),1.25 (0.40),0.86 (0.15),1.19 (0.68),1.36 (0.34),1.07 (0.34),0.93 (0.14),0.97 (0.21)
63,1.30 (0.40),1.13 (0.39),1.15 (0.33),1.25 (0.33),1.14 (0.31),2.87 (1.10),1.25 (0.26),1.27 (0.43),1.27 (0.38),0.87 (0.13),1.22 (0.66),1.39 (0.34),1.10 (0.33),0.95 (0.13),0.99 (0.21)
70,1.31 (0.37),1.15 (0.36),1.17 (0.31),1.28 (0.33),1.17 (0.31),2.97 (1.01),1.27 (0.26),1.30 (0.44),1.29 (0.36),0.87 (0.12),1.23 (0.65),1.42 (0.32),1.12 (0.30),0.96 (0.12),1.01 (0.19)


In [19]:
coverage = results_mean_std.filter(like='80')
coverage

,Trust-ssa_coverage_80,Trust-arima-hw_coverage_80,Trust-arima_coverage_80,Trust-comb_coverage_80,Trust-hw_coverage_80,Trust-fbp_coverage_80,Trust-fbp-arima-hw_coverage_80,Trust-tbats_coverage_80,Trust-ar_coverage_80,Trust-harmonic_coverage_80,Trust-reg-arima2_coverage_80,Trust-fbp-arima_coverage_80,Trust-elastic-net_coverage_80
7,0.70 (0.27),0.83 (0.21),0.81 (0.20),0.84 (0.14),0.85 (0.19),0.78 (0.20),0.82 (0.20),0.84 (0.23),0.67 (0.21),0.80 (0.23),0.78 (0.29),0.79 (0.21),0.68 (0.23)
14,0.67 (0.25),0.82 (0.19),0.78 (0.17),0.91 (0.10),0.86 (0.18),0.77 (0.17),0.81 (0.19),0.85 (0.23),0.61 (0.18),0.78 (0.20),0.76 (0.28),0.78 (0.18),0.61 (0.19)
21,0.67 (0.24),0.83 (0.17),0.77 (0.15),0.94 (0.06),0.87 (0.16),0.76 (0.14),0.82 (0.16),0.86 (0.22),0.57 (0.15),0.78 (0.15),0.74 (0.27),0.78 (0.14),0.57 (0.16)
28,0.66 (0.25),0.84 (0.14),0.76 (0.14),0.95 (0.05),0.88 (0.14),0.76 (0.12),0.83 (0.13),0.87 (0.21),0.54 (0.14),0.76 (0.14),0.71 (0.27),0.78 (0.11),0.54 (0.14)
35,0.66 (0.26),0.84 (0.14),0.75 (0.15),0.95 (0.05),0.89 (0.14),0.77 (0.11),0.83 (0.13),0.88 (0.21),0.52 (0.15),0.75 (0.16),0.69 (0.28),0.77 (0.10),0.52 (0.15)
42,0.66 (0.26),0.84 (0.14),0.74 (0.15),0.96 (0.04),0.90 (0.14),0.76 (0.10),0.84 (0.12),0.89 (0.20),0.50 (0.15),0.73 (0.16),0.66 (0.28),0.77 (0.10),0.50 (0.16)
49,0.66 (0.25),0.85 (0.13),0.73 (0.15),0.97 (0.03),0.90 (0.12),0.76 (0.10),0.84 (0.11),0.89 (0.20),0.49 (0.15),0.72 (0.17),0.64 (0.29),0.76 (0.09),0.49 (0.15)
56,0.65 (0.24),0.85 (0.12),0.73 (0.14),0.97 (0.03),0.91 (0.10),0.76 (0.10),0.84 (0.10),0.90 (0.19),0.47 (0.14),0.71 (0.18),0.63 (0.28),0.76 (0.08),0.47 (0.15)
63,0.65 (0.24),0.86 (0.12),0.72 (0.14),0.97 (0.03),0.91 (0.10),0.76 (0.09),0.84 (0.09),0.90 (0.18),0.45 (0.14),0.70 (0.18),0.62 (0.26),0.75 (0.08),0.46 (0.14)
70,0.64 (0.23),0.86 (0.11),0.72 (0.13),0.97 (0.03),0.92 (0.09),0.76 (0.09),0.85 (0.09),0.91 (0.18),0.44 (0.14),0.69 (0.18),0.61 (0.23),0.75 (0.07),0.45 (0.14)


In [20]:
coverage_80 = results_mean_std.filter(like='80')
columns = coverage_80.columns
columns = [s.replace('_coverage_80', '') for s in columns]
columns = [s.replace('Trust-', '') for s in columns]
coverage_80.columns = columns
coverage_80.sort_index(axis=1)

,ar,arima,arima-hw,comb,elastic-net,fbp,fbp-arima,fbp-arima-hw,harmonic,hw,reg-arima2,ssa,tbats
7,0.67 (0.21),0.81 (0.20),0.83 (0.21),0.84 (0.14),0.68 (0.23),0.78 (0.20),0.79 (0.21),0.82 (0.20),0.80 (0.23),0.85 (0.19),0.78 (0.29),0.70 (0.27),0.84 (0.23)
14,0.61 (0.18),0.78 (0.17),0.82 (0.19),0.91 (0.10),0.61 (0.19),0.77 (0.17),0.78 (0.18),0.81 (0.19),0.78 (0.20),0.86 (0.18),0.76 (0.28),0.67 (0.25),0.85 (0.23)
21,0.57 (0.15),0.77 (0.15),0.83 (0.17),0.94 (0.06),0.57 (0.16),0.76 (0.14),0.78 (0.14),0.82 (0.16),0.78 (0.15),0.87 (0.16),0.74 (0.27),0.67 (0.24),0.86 (0.22)
28,0.54 (0.14),0.76 (0.14),0.84 (0.14),0.95 (0.05),0.54 (0.14),0.76 (0.12),0.78 (0.11),0.83 (0.13),0.76 (0.14),0.88 (0.14),0.71 (0.27),0.66 (0.25),0.87 (0.21)
35,0.52 (0.15),0.75 (0.15),0.84 (0.14),0.95 (0.05),0.52 (0.15),0.77 (0.11),0.77 (0.10),0.83 (0.13),0.75 (0.16),0.89 (0.14),0.69 (0.28),0.66 (0.26),0.88 (0.21)
42,0.50 (0.15),0.74 (0.15),0.84 (0.14),0.96 (0.04),0.50 (0.16),0.76 (0.10),0.77 (0.10),0.84 (0.12),0.73 (0.16),0.90 (0.14),0.66 (0.28),0.66 (0.26),0.89 (0.20)
49,0.49 (0.15),0.73 (0.15),0.85 (0.13),0.97 (0.03),0.49 (0.15),0.76 (0.10),0.76 (0.09),0.84 (0.11),0.72 (0.17),0.90 (0.12),0.64 (0.29),0.66 (0.25),0.89 (0.20)
56,0.47 (0.14),0.73 (0.14),0.85 (0.12),0.97 (0.03),0.47 (0.15),0.76 (0.10),0.76 (0.08),0.84 (0.10),0.71 (0.18),0.91 (0.10),0.63 (0.28),0.65 (0.24),0.90 (0.19)
63,0.45 (0.14),0.72 (0.14),0.86 (0.12),0.97 (0.03),0.46 (0.14),0.76 (0.09),0.75 (0.08),0.84 (0.09),0.70 (0.18),0.91 (0.10),0.62 (0.26),0.65 (0.24),0.90 (0.18)
70,0.44 (0.14),0.72 (0.13),0.86 (0.11),0.97 (0.03),0.45 (0.14),0.76 (0.09),0.75 (0.07),0.85 (0.09),0.69 (0.18),0.92 (0.09),0.61 (0.23),0.64 (0.23),0.91 (0.18)


In [28]:
coverage_80_2 = coverage_80[ordered_cols[1:]]
coverage_80_2.columns = model_nos[1:]
coverage_80_2

KeyError: "['reg-arima'] not in index"

In [21]:
coverage_95 = results_mean_std.filter(like='95')
columns = coverage_95.columns
columns = [s.replace('_coverage_95', '') for s in columns]
columns = [s.replace('Trust-', '') for s in columns]
coverage_95.columns = columns
coverage_95.sort_index(axis=1)

,ar,arima,arima-hw,comb,elastic-net,fbp,fbp-arima,fbp-arima-hw,harmonic,hw,reg-arima,reg-arima2,ssa,tbats
7,0.90 (0.15),0.96 (0.09),0.97 (0.07),0.96 (0.08),0.89 (0.16),0.92 (0.14),0.94 (0.13),0.94 (0.13),0.94 (0.15),0.97 (0.07),0.96 (0.08),0.93 (0.16),0.90 (0.19),0.97 (0.06)
14,0.86 (0.16),0.95 (0.11),0.95 (0.11),0.97 (0.05),0.85 (0.17),0.91 (0.11),0.93 (0.11),0.92 (0.14),0.94 (0.12),0.96 (0.11),0.94 (0.11),0.91 (0.18),0.91 (0.16),0.97 (0.08)
21,0.83 (0.15),0.95 (0.10),0.95 (0.11),0.98 (0.04),0.82 (0.15),0.91 (0.09),0.92 (0.10),0.93 (0.11),0.94 (0.10),0.96 (0.10),0.93 (0.11),0.90 (0.19),0.91 (0.15),0.97 (0.06)
28,0.80 (0.14),0.94 (0.09),0.95 (0.09),0.99 (0.03),0.79 (0.13),0.91 (0.08),0.92 (0.08),0.94 (0.09),0.94 (0.09),0.97 (0.08),0.93 (0.10),0.89 (0.20),0.91 (0.14),0.98 (0.05)
35,0.78 (0.14),0.94 (0.09),0.95 (0.08),0.99 (0.02),0.77 (0.14),0.91 (0.07),0.92 (0.07),0.94 (0.08),0.93 (0.09),0.97 (0.07),0.92 (0.10),0.89 (0.19),0.91 (0.14),0.98 (0.04)
42,0.75 (0.15),0.94 (0.08),0.96 (0.07),0.99 (0.02),0.75 (0.14),0.91 (0.07),0.92 (0.06),0.94 (0.07),0.92 (0.09),0.98 (0.05),0.91 (0.10),0.88 (0.19),0.91 (0.14),0.98 (0.03)
49,0.73 (0.16),0.93 (0.08),0.96 (0.06),0.99 (0.02),0.74 (0.14),0.91 (0.06),0.92 (0.06),0.95 (0.06),0.91 (0.09),0.98 (0.05),0.91 (0.09),0.87 (0.19),0.91 (0.14),0.98 (0.03)
56,0.71 (0.16),0.93 (0.08),0.96 (0.06),0.99 (0.01),0.72 (0.13),0.91 (0.06),0.92 (0.05),0.95 (0.06),0.90 (0.10),0.98 (0.04),0.90 (0.08),0.88 (0.16),0.91 (0.14),0.98 (0.02)
63,0.70 (0.16),0.93 (0.07),0.96 (0.05),0.99 (0.01),0.71 (0.13),0.90 (0.06),0.92 (0.04),0.95 (0.05),0.89 (0.11),0.98 (0.04),0.89 (0.09),0.87 (0.16),0.91 (0.12),0.99 (0.02)
70,0.68 (0.16),0.92 (0.06),0.96 (0.05),0.99 (0.01),0.70 (0.13),0.90 (0.05),0.92 (0.04),0.95 (0.05),0.88 (0.12),0.98 (0.03),0.89 (0.09),0.87 (0.15),0.91 (0.11),0.99 (0.02)


In [22]:
coverage_95_2 = coverage_95[ordered_cols[1:]]
coverage_95_2.columns = model_nos[1:]
coverage_95_2

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
7,0.97 (0.07),0.96 (0.09),0.90 (0.15),0.96 (0.08),0.93 (0.16),0.89 (0.16),0.94 (0.15),0.90 (0.19),0.92 (0.14),0.97 (0.06),0.96 (0.08),0.97 (0.07),0.94 (0.13),0.94 (0.13)
14,0.96 (0.11),0.95 (0.11),0.86 (0.16),0.94 (0.11),0.91 (0.18),0.85 (0.17),0.94 (0.12),0.91 (0.16),0.91 (0.11),0.97 (0.08),0.97 (0.05),0.95 (0.11),0.93 (0.11),0.92 (0.14)
21,0.96 (0.10),0.95 (0.10),0.83 (0.15),0.93 (0.11),0.90 (0.19),0.82 (0.15),0.94 (0.10),0.91 (0.15),0.91 (0.09),0.97 (0.06),0.98 (0.04),0.95 (0.11),0.92 (0.10),0.93 (0.11)
28,0.97 (0.08),0.94 (0.09),0.80 (0.14),0.93 (0.10),0.89 (0.20),0.79 (0.13),0.94 (0.09),0.91 (0.14),0.91 (0.08),0.98 (0.05),0.99 (0.03),0.95 (0.09),0.92 (0.08),0.94 (0.09)
35,0.97 (0.07),0.94 (0.09),0.78 (0.14),0.92 (0.10),0.89 (0.19),0.77 (0.14),0.93 (0.09),0.91 (0.14),0.91 (0.07),0.98 (0.04),0.99 (0.02),0.95 (0.08),0.92 (0.07),0.94 (0.08)
42,0.98 (0.05),0.94 (0.08),0.75 (0.15),0.91 (0.10),0.88 (0.19),0.75 (0.14),0.92 (0.09),0.91 (0.14),0.91 (0.07),0.98 (0.03),0.99 (0.02),0.96 (0.07),0.92 (0.06),0.94 (0.07)
49,0.98 (0.05),0.93 (0.08),0.73 (0.16),0.91 (0.09),0.87 (0.19),0.74 (0.14),0.91 (0.09),0.91 (0.14),0.91 (0.06),0.98 (0.03),0.99 (0.02),0.96 (0.06),0.92 (0.06),0.95 (0.06)
56,0.98 (0.04),0.93 (0.08),0.71 (0.16),0.90 (0.08),0.88 (0.16),0.72 (0.13),0.90 (0.10),0.91 (0.14),0.91 (0.06),0.98 (0.02),0.99 (0.01),0.96 (0.06),0.92 (0.05),0.95 (0.06)
63,0.98 (0.04),0.93 (0.07),0.70 (0.16),0.89 (0.09),0.87 (0.16),0.71 (0.13),0.89 (0.11),0.91 (0.12),0.90 (0.06),0.99 (0.02),0.99 (0.01),0.96 (0.05),0.92 (0.04),0.95 (0.05)
70,0.98 (0.03),0.92 (0.06),0.68 (0.16),0.89 (0.09),0.87 (0.15),0.70 (0.13),0.88 (0.12),0.91 (0.11),0.90 (0.05),0.99 (0.02),0.99 (0.01),0.96 (0.05),0.92 (0.04),0.95 (0.05)


In [23]:
#Write these rather large tables to a latex file to use in appendix

#Table S1: MASE
pf_mase2.to_latex(buf=f'{APPENDIX_PATH}tableS1.tex')

#Table S2: 80% Coverage
coverage_80.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}tableS2.tex')

#Table S3: 95% coverage
coverage_95_2.to_latex(buf=f'{APPENDIX_PATH}tableS3.tex')

#extra rables for sMAPE and RMSE - not used in paper.
pf_smape.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}symmetricmape.tex')
pf_rmse.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}rootmeansqua.tex')


print(f'Appendix summary tables written to {APPENDIX_PATH}')

Appendix summary tables written to ../paper/appendix/


## Naive Benchmark (Seasonal Naive)

A naive baseline forecasting method was chosen. This was to ensure that the sophisticated methods we test in the study were only considered for the final benchmark if they provided more more accurate point forecasts than the simplest of models. As emergency care demand data are seasonal we opted for the well-known Seasonal Naive method.  This method works by using the most recent observation for the same day and carrying it forward.  For example, if we are forecasting next Tuesday then the observation from the most recent Tuesday is used as the predicted value.

In [24]:
pf_snaive = point_forecasts.filter(like='snaive')
pf_snaive.reindex(sorted(pf_snaive.columns), axis=1)
columns = pf_snaive.columns
columns = [s.replace('_rmse', '') for s in columns]
columns = [s.replace('_smape', '') for s in columns]
columns = [s.replace('_mase', '') for s in columns]
pf_snaive.columns = ['sMAPE', 'RMSE', 'MASE']

In [25]:
pf_snaive[['MASE', 'sMAPE', 'RMSE']].to_latex(buf=f'{TABLE_PATH}table2.tex')

In [26]:
pf_snaive[:-1]

,sMAPE,RMSE,MASE
7,3.48 (1.19),96.91 (37.11),0.94 (0.35)
14,3.94 (1.32),109.71 (41.88),1.06 (0.39)
21,4.12 (1.38),115.38 (42.94),1.12 (0.41)
28,4.19 (1.33),118.34 (42.92),1.14 (0.40)
35,4.35 (1.34),123.26 (42.33),1.18 (0.40)
42,4.51 (1.33),128.11 (40.34),1.23 (0.40)
49,4.61 (1.32),131.40 (39.79),1.25 (0.39)
56,4.67 (1.33),133.95 (40.20),1.27 (0.39)
63,4.77 (1.36),136.59 (40.30),1.30 (0.40)
70,4.82 (1.26),138.70 (37.25),1.31 (0.37)


In [27]:
print('\nTable 2: Cross-Validation of Seasonal Naive Point Forecasts')
print(pf_snaive[:-1])


Table 2: Cross-Validation of Seasonal Naive Point Forecasts
          sMAPE            RMSE         MASE
7   3.48 (1.19)   96.91 (37.11)  0.94 (0.35)
14  3.94 (1.32)  109.71 (41.88)  1.06 (0.39)
21  4.12 (1.38)  115.38 (42.94)  1.12 (0.41)
28  4.19 (1.33)  118.34 (42.92)  1.14 (0.40)
35  4.35 (1.34)  123.26 (42.33)  1.18 (0.40)
42  4.51 (1.33)  128.11 (40.34)  1.23 (0.40)
49  4.61 (1.32)  131.40 (39.79)  1.25 (0.39)
56  4.67 (1.33)  133.95 (40.20)  1.27 (0.39)
63  4.77 (1.36)  136.59 (40.30)  1.30 (0.40)
70  4.82 (1.26)  138.70 (37.25)  1.31 (0.37)
77  4.88 (1.18)  140.73 (34.49)  1.33 (0.34)
84  4.94 (1.21)  143.04 (34.07)  1.34 (0.35)
